# Snippets for GP app

In [20]:
# === GP Snippet Creator with Integrated Search ===

import os
import json
import time
import subprocess
import pyautogui
import clipboard
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI


In [21]:

## === load environment and initialize api === part3
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()
MODEL = "gpt-4o"


In [22]:
# === Data Storage === part 3
DATA_FILE = "snippets.json"

def load_snippets():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_snippets(snippets):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(snippets, f, indent=2)

def add_snippet(label, icpc_code, language, content, shortcut):
    snippets = load_snippets()
    snippets.append({
        "label": label.strip(),
        "icpc_code": icpc_code.upper().strip(),
        "language": language,
        "content": content.strip(),
        "shortcut": shortcut.strip().lower()
    })
    save_snippets(snippets)
    return f"✅ Snippet '{label}' saved."


In [23]:
# === Format & List === part 3
def format_snippet(snippet):
    return f"{snippet.get('shortcut', '')} → {snippet.get('content', '')}"

def list_snippets():
    return load_snippets()

def update_list():
    return [format_snippet(s) for s in load_snippets()]

In [24]:
## === export snippets to AHK file === part 3
def export_snippets_to_ahk():
    snippets = list_snippets()
    lines = ["#SingleInstance force", "; AutoHotKey Snippets for GP"]
    for s in snippets:
        sc = s.get("shortcut", "").strip()
        ct = ' '.join(s.get("content", "").split())
        if sc and ct:
            lines.append(f"::{sc}::{ct}")
    with open("snippets.ahk", "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    return "✅ Exported AHK snippets."

def export_snippets_and_run():
    export_snippets_to_ahk()
    try:
        subprocess.Popen([r"C:\\Program Files\\AutoHotkey\\v1.1.37.02\\AutoHotkeyU64.exe", "snippets.ahk"], shell=True)
        return "✅ AutoHotkey script launched."
    except Exception as e:
        return f"❌ Failed to run AHK: {e}"


In [25]:
# === Update AHK Popup === part 3
def send_to_popup(ahk_text):
    clipboard.copy(ahk_text)
    time.sleep(0.2)
    try:
        window = pyautogui.getWindowsWithTitle("Snippet Popup")[0]
        window.activate()
        time.sleep(0.2)
        pyautogui.hotkey("ctrl", "a")
        pyautogui.hotkey("ctrl", "v")
    except IndexError:
        print("❌ Could not find the AHK popup window titled 'Snippet Popup'.")

def export_snippets_for_popup(file_path="snippets_data.ahk"):
    snippets = load_snippets()
    lines = []
    for snip in snippets:
        sc = snip.get("shortcut", "").strip()
        ct = snip.get("content", "").strip()
        if sc and ct:
            lines.append(f"{sc}::{ct}")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))


In [26]:
## === AI snippet generator === part 3
def generate_snippet_with_ai(label, icpc_code, language):
    prompt = f"You are an experienced doctor with 20 years of medical knowledge. Create a short, concise SOAP Plan (P:) snippet to document a consultation for '{label}' (ICPC: {icpc_code}) in {language}. The snippet should be suitable for efficient note-taking in a medical record."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful medical assistant generating efficient SOAP Plan snippets for busy general practitioners."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [27]:
## === ai short snippet generator (for EHR) === part 3
def generate_short_snippet(label, icpc_code, language):
    prompt = f"You are an experienced doctor with 20 years of medical knowledge. Write a *very short*, structured SOAP Plan (P:) in {language} for '{label}' (ICPC: {icpc_code}) suitable for directly pasting into an EHR. Avoid markdown and formatting."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You write short SOAP notes suitable for EHR input, using plain text."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [28]:
## === Gradio interface === part 3
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 GP Snippet Creator with AI Support")

    with gr.Row():
        label = gr.Textbox(label="Snippet Label", placeholder="e.g. Cough")
        icpc = gr.Textbox(label="ICPC Code", placeholder="e.g. R05")
        lang = gr.Radio(["EN", "NL"], label="Language")

    shortcut = gr.Textbox(label="Shortcut Trigger", placeholder="e.g. ;cough")
    content = gr.Textbox(label="Snippet Content", lines=6)

    with gr.Row():
        generate_btn = gr.Button("💡 Generate Snippet with AI")
        shorten_btn = gr.Button("✂️ Generate Short Version")
        add_btn = gr.Button("➕ Save Snippet")

    with gr.Row():
        export_btn = gr.Button("📤 Export to .ahk")
        run_and_export_btn = gr.Button("🚀 Save & Run")

    status = gr.Textbox(label="Status", interactive=False)
    show_snippets = gr.Checkbox(label="Show Saved Snippets", value=False)
    search_query = gr.Textbox(label="Search Snippets")
    snippet_list = gr.List(label="Saved Snippets", interactive=False, visible=False)

    def handle_add(label, icpc_code, language, content, shortcut):
        add_snippet(label, icpc_code, language, content, shortcut)
        export_snippets_for_popup()
        export_snippets_to_ahk()
        export_snippets_and_run()
        return "✅ Snippet saved and popup updated!", update_list()

    def toggle_snippet_list(show):
        return gr.update(visible=show)

    def search_snippets(query):
        snippets = list_snippets()
        return [format_snippet(s) for s in snippets if query.lower() in s.get("label", "").lower() or query.lower() in s.get("shortcut", "").lower() or query.lower() in s.get("content", "").lower()]

    add_btn.click(fn=handle_add, inputs=[label, icpc, lang, content, shortcut], outputs=[status, snippet_list])
    export_btn.click(fn=export_snippets_to_ahk, outputs=[status])
    run_and_export_btn.click(fn=export_snippets_and_run, outputs=[status])
    generate_btn.click(fn=generate_snippet_with_ai, inputs=[label, icpc, lang], outputs=content)
    shorten_btn.click(fn=generate_short_snippet, inputs=[label, icpc, lang], outputs=content)
    show_snippets.change(fn=toggle_snippet_list, inputs=show_snippets, outputs=snippet_list)
    search_query.change(fn=search_snippets, inputs=search_query, outputs=snippet_list)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
